# Probability (Continuous)

In [1]:
# Import some helper functions (please ignore this!)
from utils import * 

**Context:** Many real-world data sets include non-discrete values (e.g. a patient's body-mass index (BMI), the dosage of medicine, and more). Here, we will introduce what you need to know in order to model continuous-valued data. 

**Challenge:** In many ways, continuous probability is similar to discrete probability. However, there are a few "gotchas" that are important to highlight. 

**Outline:**
* Introduce and practice the concepts, terminology, and notation behind continuous probability distributions.
* Gain familiarity with several common continuous distributions.

## Differences Between Continuous and Discrete Probability

Continuous probability functions the same way as discrete probability, except for a few key differences.

**Sample Space of Support:** The support of continuous probability distributions is over "uncountably infinite sets." If you're not familiar with this term, that's ok! In this course, we'll think about it more as a distribution supported over the real numbers, $\mathbb{R}$ (or some subset thereof). 
> Example: Let $H$ be a continuous RV, describing the distribution of heights in the US. The support of $H$ is the interval $(0, \infty)$ on the real line. 

**Probability Mass Function (PMF):** Continuous probability distributions *do not have PMFs*. 



**Cumulative Density Function (CDF):**

**Probability Density Function (PDF):**

```{admonition} Exercise: Gaining comfort with commonly-used continuous distributions
Browse the Wikipedia pages for the following distributions:
* [Uniform](https://en.wikipedia.org/wiki/Continuous_uniform_distribution)
* [Beta](https://en.wikipedia.org/wiki/Beta_distribution)
* [Normal](https://en.wikipedia.org/wiki/Normal_distribution)
* [Truncated Normal](https://en.wikipedia.org/wiki/Truncated_normal_distribution)
* [Laplace](https://en.wikipedia.org/wiki/Laplace_distribution)

Then, for each distribution,
1. Describe a random phenomenon from everyday life that can be described using the distribution. 
2. Define a random variable for the phenomenon you described. Write the support and PMF for this random variable using the above notation.
3. Describe a setting of the distribution's parameters that makes sense for your specific phenomenon (and explain why it makes sense).

*Hint: On each Wikipedia page, there's a panel on the right side that summarizes the properties of the distribution (e.g. its support, PDF, example plots, etc.).*
```